In [9]:
%load_ext autoreload
%autoreload 2

import shutil
import os
import inspect
import sys
sys.path.append("/home/msst/repo/Quantization")

import torch
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer

import qlib


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path_to_fp_model = "/media/msst/ssd_storage1/ml/llm/pretrained_models/Llama2-7B"
path_to_q_config = "/home/msst/repo/Quantization/experiments/Llama2-7B/Llama2-7B_w8a8/hfconfig_Llama2-7B_w8a8.json"

In [3]:
fp_model = AutoModelForCausalLM.from_pretrained(
    path_to_fp_model,
    device_map="cpu",
    dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained(path_to_fp_model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 63.18it/s]


In [4]:
q_model_config = AutoConfig.from_pretrained(path_to_q_config)
q_model_class = getattr(qlib, q_model_config.architectures[0])
q_model = q_model_class(q_model_config)

LOL, I'm quantize LlamaModel


In [5]:
with torch.no_grad():
    for q_module_name, q_module in q_model.named_modules():
        if isinstance(q_module, qlib.W8A8Linear):
            fp_module = fp_model.get_submodule(q_module_name)
            q_module.configure(
                fp_module=fp_module, 
                device=torch.device(0), 
                verbose=True, 
                module_name=q_module_name
            )
        elif hasattr(q_module, 'weight'):
            fp_module = fp_model.get_submodule(q_module_name)
            q_module.weight.data.copy_(fp_module.weight.data)
        q_module.to("cpu")


model.layers.0.self_attn.q_proj
error (w-wq)^2: 0.001
model.layers.0.self_attn.k_proj
error (w-wq)^2: 0.001
model.layers.0.self_attn.v_proj
error (w-wq)^2: 0.000
model.layers.0.self_attn.o_proj
error (w-wq)^2: 0.000
model.layers.0.mlp.gate_proj
error (w-wq)^2: 0.000
model.layers.0.mlp.up_proj
error (w-wq)^2: 0.000
model.layers.0.mlp.down_proj
error (w-wq)^2: 0.000
model.layers.1.self_attn.q_proj
error (w-wq)^2: 0.000
model.layers.1.self_attn.k_proj
error (w-wq)^2: 0.000
model.layers.1.self_attn.v_proj
error (w-wq)^2: 0.000
model.layers.1.self_attn.o_proj
error (w-wq)^2: 0.000
model.layers.1.mlp.gate_proj
error (w-wq)^2: 0.000
model.layers.1.mlp.up_proj
error (w-wq)^2: 0.000
model.layers.1.mlp.down_proj
error (w-wq)^2: 0.000
model.layers.2.self_attn.q_proj
error (w-wq)^2: 0.000
model.layers.2.self_attn.k_proj
error (w-wq)^2: 0.000
model.layers.2.self_attn.v_proj
error (w-wq)^2: 0.000
model.layers.2.self_attn.o_proj
error (w-wq)^2: 0.000
model.layers.2.mlp.gate_proj
error (w-wq)^2: 0.000

In [6]:
for param_name, param in q_model.named_parameters():
    if "matmul_out_scale" not in param_name:
        param.data = param.data.to(torch.float16)

for param_name, param in q_model.named_parameters():
    print(param_name, param.dtype)

model.embed_tokens.weight torch.float16
model.layers.0.self_attn.q_proj.weight_scale torch.float16
model.layers.0.self_attn.q_proj.matmul_out_scale torch.float32
model.layers.0.self_attn.k_proj.weight_scale torch.float16
model.layers.0.self_attn.k_proj.matmul_out_scale torch.float32
model.layers.0.self_attn.v_proj.weight_scale torch.float16
model.layers.0.self_attn.v_proj.matmul_out_scale torch.float32
model.layers.0.self_attn.o_proj.weight_scale torch.float16
model.layers.0.self_attn.o_proj.matmul_out_scale torch.float32
model.layers.0.mlp.gate_proj.weight_scale torch.float16
model.layers.0.mlp.gate_proj.matmul_out_scale torch.float32
model.layers.0.mlp.up_proj.weight_scale torch.float16
model.layers.0.mlp.up_proj.matmul_out_scale torch.float32
model.layers.0.mlp.down_proj.weight_scale torch.float16
model.layers.0.mlp.down_proj.matmul_out_scale torch.float32
model.layers.0.input_layernorm.weight torch.float16
model.layers.0.post_attention_layernorm.weight torch.float16
model.layers.1.

In [10]:
### Save qmodel as hf model
save_path = "/home/msst/repo/Quantization/logs/Llama2-7B_w8a8"

q_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
try:
    modeling_file = inspect.getfile(q_model_class)
    dst_file = os.path.join(save_path, os.path.basename(modeling_file))
    shutil.copy(modeling_file, dst_file)
except:
    print(f"WARNING: failed to save modeling to \"{save_path}\"!")